In [ ]:
# Stable Diffusion 설치
!pip install --upgrade -qq git+https://github.com/huggingface/diffusers.git transformers accelerate

In [ ]:
import torch
from diffusers import StableDiffusionControlNetPipeline
from diffusers.utils import load_image

In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
import cv2
from PIL import Image
import numpy as np

# 이미지를 NumPy 배열로 변환합니다.
img = np.array(img)

# threshold를 지정합니다.
low_threshold = 100
high_threshold = 200

# 윤곽선을 검출합니다.
image = cv2.Canny(img, low_threshold, high_threshold)
image = image[:, :, None]
image = np.concatenate([image, image, image], axis=2)
canny_image = Image.fromarray(image)  # NumPy 배열을 PIL 이미지로 변환합니다.

canny_image

In [ ]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel

canny_controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16)
canny_pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=canny_controlnet, torch_dtype=torch.float16
)

In [ ]:
from diffusers import UniPCMultistepScheduler
canny_pipe.scheduler = UniPCMultistepScheduler.from_config(canny_pipe.scheduler.config)
canny_pipe = canny_pipe.to("cuda")

generator = torch.manual_seed(0)

canny_image = canny_pipe(
    prompt="robots are sitting on the box",
    num_inference_steps=20,
    generator=generator,
    image=canny_image
).images[0]

In [ ]:
canny_image

뭐지... 이.. 난해한 그림은

In [ ]:
!pip install controlnet-aux==0.0.1

In [ ]:
from diffusers.utils import load_image

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
from PIL import Image
img = Image.open('IMG_0271-26040.jpg')
img

In [ ]:
from controlnet_aux import OpenposeDetector

# 인체의 자세를 검출하는 사전 학습된 ControlNet 불러오기
openpose = OpenposeDetector.from_pretrained("lllyasviel/ControlNet")

# 이미지에서 자세 검출
openpose_image = openpose(img)
openpose_image

In [ ]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel

openpose_controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-openpose", torch_dtype=torch.float16)
openpose_pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=openpose_controlnet, torch_dtype=torch.float16
)

In [ ]:
from diffusers import UniPCMultistepScheduler
import torch

# 스케줄러 설정
openpose_pipe.scheduler = UniPCMultistepScheduler.from_config(openpose_pipe.scheduler.config)

# CUDA로 모델 이동
openpose_pipe = openpose_pipe.to("cuda")

# ✅ 시드 고정 (동일한 결과 생성)
generator = torch.Generator(device="cuda").manual_seed(42)

# ✅ 프롬프트 설정
prompt = "a person doing a dynamic pose, full body, detailed clothing, high quality, professional photo"
negative_prompt = "low quality, blurry, distorted, extra limbs"

# ✅ 입력 이미지
images = openpose_image  # 보통 openpose 이미지에서 받아온 입력 이미지일 거야

# ✅ 이미지 생성
openpose_image1 = openpose_pipe(
    prompt=prompt,
    image=images,
    negative_prompt=negative_prompt,
    generator=generator
).images[0]

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(openpose_image1)
plt.axis("off")
plt.show()

무서워.... 손이 이상해.....

In [ ]:
from diffusers.utils import load_image
from PIL import Image
import cv2
import numpy as np

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
from PIL import Image
img = Image.open('IMG_0273-26042.jpg')
img

In [ ]:
low_threshold = 100
high_threshold = 200

In [ ]:
canny_image = np.array(img)

In [ ]:
canny_image= cv2.Canny(canny_image, low_threshold, high_threshold)



In [ ]:
zero_start = canny_image.shape[1] // 4
zero_end = zero_start + canny_image.shape[1] // 2
canny_image[:, zero_start:zero_end] = 0

In [ ]:
canny_image = np.stack([canny_image]*3, axis=-1)

In [ ]:
canny_image = Image.fromarray(canny_image)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(canny_image)
plt.axis("off")
plt.show()

In [ ]:
from controlnet_aux import OpenposeDetector
from diffusers.utils import load_image

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
from PIL import Image
img = Image.open('IMG_0319-26088.jpg')
img

In [ ]:
openpose = OpenposeDetector.from_pretrained("lllyasviel/ControlNet")

In [ ]:
openpose_image = openpose(img)

In [ ]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
import torch

In [ ]:
controlnets = [
    ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-openpose", torch_dtype=torch.float16),
    ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16)
]

In [ ]:
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnets,
    torch_dtype=torch.float16
)

In [ ]:
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")

In [ ]:
prompt = "two cats are sitting on the box, 4k, detailed"
negative_prompt = "low quality, blurry, distorted, extra limbs, bad anatomy"

In [ ]:
generator = torch.Generator(device="cuda").manual_seed(42)


In [ ]:
images = [openpose_image, canny_image]


In [ ]:
from PIL import Image

# 이미지 크기를 동일하게 맞춰줌 (매우 중요!)
openpose_image = openpose_image.resize((512, 512))
canny_image = canny_image.resize((512, 512))

images = [openpose_image, canny_image]

# 파이프라인 실행
image = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    image=images,
    generator=generator,
    num_inference_steps=30
).images[0]

In [ ]:
image

ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ

#회고

* 재미있었다! 내가 설정한대로 나오진 않지만 ㅋㅋㅋㅋㅋㅋㅋㅋ
* 프롬프트를 짜는 기술의 중요성이 높아질 것 같다.
* 마지막 고양이 짤이 제일 웃기다 ㅋㅋ